## Importing libraries and modules

In [1]:
# Enable auto-reload
%load_ext autoreload
%autoreload 2

In [2]:
# Suppress warnings
import warnings
warnings.filterwarnings("ignore")

# Add one level up directory to the path
import sys
sys.path.append("..")

# Import libraries
from src.data import *
from src.data.utils import get_dataset
from src.data.preprocess import clean_text
from src.data.tokenizer import CustomBartTokenizer
from src.machine_translation.utils import get_data_loader_models

## Data acquisition, cleaning and processing

In [3]:
%%time
train_df, validation_df, test_df = get_dataset()

CPU times: user 4.31 s, sys: 164 ms, total: 4.48 s
Wall time: 7.58 s


In [4]:
train_df

,hi_en,en
0,film ka kya naam hai,What's the name of the movie
1,"namaste, sada hua tomatoes score mahaan hai, l...","Hi, the rotten tomatoes score is great but the..."
2,kya aapako lagata hai ki aapako film pasand aa...,Do you think you will like the movie
3,yah kis tarah kee philm hai,What kind of movie is it
4,film kab banee thee?,when was the movie made?
...,...,...
8055,Thik hai\n,Ok.
8056,Thik hai bhai\n,ok bro
8057,Kya ham chalu kar sakte hai?\n,shall we continue?
8058,Kya aapko pasand hai hamare saath\n,do you like we can


In [5]:
validation_df

,hi_en,en
0,movie kis baare me hai?,What is the movie about?
1,Movie ek chhote bacche Kevin k baare me hai ji...,the movie is about a young child named Kevin w...
2,Kya wo jaan bhuj k abandon karte hai?,Did they abandon him on purpose?
3,"nahi, wo uska track lose kardete hai kyuki bah...",no they had lost track of him since they had m...
4,Kya wo realize karte hai k wo chhut gaya aur u...,Did they realize they lost track of him and co...
...,...,...
936,Sunkar good movie hai. Kya ham finish kar skat...,Sounds like a good movie. Can we finish now?
937,Yep. Thanks baat karne ke liye\n,Yep. Thanks for chatting
938,"Thanks, mei dekhati hui. Achi baat hai\n","thanks, I will watch it. SOunds good"
940,kya tumhe movie Despicable Me pasand hai?\n,Did you like the movie Despicable Me?


In [6]:
test_df

,hi_en,en
0,oye ledki kitni mazakiya movie dekh rhi ho ?,ah mean girls. such a funny movie. have you se...
1,ha muje bhi hassi mazak ki movies pasand par y...,"Yeah, even though I love comedies, this wasn't..."
2,kese tina ka jhuti writing isme achi the usne ...,how come. thought tina fey's writing was fanta...
3,muje kahani bhute achi lagi aur unhone ise dac...,I loved the story & how true they made it in h...
4,ab samja muje nhi pta tha ki aap sahi admi hai...,I see. i didn't realize it was 14 years ago. y...
...,...,...
6508,alarm ko abhi stop kare,Stop alarm now
6509,Har ghante ke liye alarm set kare,set alarm every hour
6510,Bobby ko text kare,text Bobby
6511,Muje shaam 6 baje laundry ko pick up karne ke ...,remind me to pick up laundry at 6 pm


In [7]:
%%time
train_df = train_df.applymap(clean_text)
train_df

CPU times: user 2.09 s, sys: 1.51 ms, total: 2.09 s
Wall time: 2.1 s


,hi_en,en
0,film ka kya naam hai,what's the name of the movie
1,"namaste, sada hua tomatoes score mahaan hai, l...","hi, the rotten tomatoes score is great but the..."
2,kya aapako lagata hai ki aapako film pasand aa...,do you think you will like the movie
3,yah kis tarah kee philm hai,what kind of movie is it
4,film kab banee thee?,when was the movie made?
...,...,...
8055,thik hai,ok.
8056,thik hai bhai,ok bro
8057,kya ham chalu kar sakte hai?,shall we continue?
8058,kya aapko pasand hai hamare saath,do you like we can


In [8]:
%%time
validation_df = validation_df.applymap(clean_text)
validation_df

CPU times: user 42.3 ms, sys: 553 µs, total: 42.9 ms
Wall time: 42.4 ms


,hi_en,en
0,movie kis baare me hai?,what is the movie about?
1,movie ek chhote bacche kevin k baare me hai ji...,the movie is about a young child named kevin w...
2,kya wo jaan bhuj k abandon karte hai?,did they abandon him on purpose?
3,"nahi, wo uska track lose kardete hai kyuki bah...",no they had lost track of him since they had m...
4,kya wo realize karte hai k wo chhut gaya aur u...,did they realize they lost track of him and co...
...,...,...
936,sunkar good movie hai. kya ham finish kar skat...,sounds like a good movie. can we finish now?
937,yep. thanks baat karne ke liye,yep. thanks for chatting
938,"thanks, mei dekhati hui. achi baat hai","thanks, i will watch it. sounds good"
940,kya tumhe movie despicable me pasand hai?,did you like the movie despicable me?


In [9]:
%%time
test_df = test_df.applymap(clean_text)
test_df

CPU times: user 83.9 ms, sys: 661 µs, total: 84.5 ms
Wall time: 84.1 ms


,hi_en,en
0,oye ledki kitni mazakiya movie dekh rhi ho ?,ah mean girls. such a funny movie. have you se...
1,ha muje bhi hassi mazak ki movies pasand par y...,"yeah, even though i love comedies, this wasn't..."
2,kese tina ka jhuti writing isme achi the usne ...,how come. thought tina fey's writing was fanta...
3,muje kahani bhute achi lagi aur unhone ise dac...,i loved the story & how true they made it in h...
4,ab samja muje nhi pta tha ki aap sahi admi hai...,i see. i didn't realize it was 14 years ago. y...
...,...,...
6508,alarm ko abhi stop kare,stop alarm now
6509,har ghante ke liye alarm set kare,set alarm every hour
6510,bobby ko text kare,text bobby
6511,muje shaam 6 baje laundry ko pick up karne ke ...,remind me to pick up laundry at 6 pm


## Data tokenization

In [10]:
bart_tokenizer = CustomBartTokenizer().build()
bart_tokenizer_scratch = CustomBartTokenizer().build(data=train_df["hi_en"], tokenizer_style=STYLE.SCRATCH.value)
bart_tokenizer_append = CustomBartTokenizer().build(data=train_df["hi_en"], tokenizer_style=STYLE.APPEND.value)

In [11]:
query = "Hello! Ye CSCI 544 ka project he. Ye project Code-Mixed Machine Translation ke bara me he. Hamane hamari khoon bahahe."
query = clean_text(text=query)

In [12]:
token_idx = bart_tokenizer.encode(query, add_special_tokens=True)
token_word = bart_tokenizer.convert_ids_to_tokens(token_idx)
print('#'*100)
print("This is an example of using the default Bart Tokenizer.\n")
print(f"Sentence: {query}\n")
print(f"Tokenized word: {token_word}\n")
print(f"Tokenized idx: {token_idx}")
print('#'*100)

####################################################################################################
This is an example of using the default Bart Tokenizer.

Sentence: hello! ye csci 544 ka project he. ye project code-mixed machine translation ke bara me he. hamane hamari khoon bahahe.

Tokenized word: ['<s>', 'hello', '!', 'Ġye', 'Ġc', 'sci', 'Ġ5', '44', 'Ġka', 'Ġproject', 'Ġhe', '.', 'Ġye', 'Ġproject', 'Ġcode', '-', 'm', 'ixed', 'Ġmachine', 'Ġtranslation', 'Ġke', 'Ġbar', 'a', 'Ġme', 'Ġhe', '.', 'Ġham', 'ane', 'Ġham', 'ari', 'Ġkh', 'oon', 'Ġb', 'aha', 'he', '.', '</s>']

Tokenized idx: [0, 42891, 328, 32440, 740, 43428, 195, 3305, 4661, 695, 37, 4, 32440, 695, 3260, 12, 119, 24194, 3563, 19850, 7321, 2003, 102, 162, 37, 4, 11402, 1728, 11402, 1512, 16447, 3863, 741, 11695, 700, 4, 2]
####################################################################################################


In [13]:
token_idx = bart_tokenizer_scratch.encode(query, add_special_tokens=True)
token_word = bart_tokenizer_scratch.convert_ids_to_tokens(token_idx)
print('#'*100)
print("This is an example of the Bart Tokenizer that is trained from scratch.\n")
print(f"Sentence: {query}\n")
print(f"Tokenized word: {token_word}\n")
print(f"Tokenized idx: {token_idx}")
print('#'*100)

####################################################################################################
This is an example of the Bart Tokenizer that is trained from scratch.

Sentence: hello! ye csci 544 ka project he. ye project code-mixed machine translation ke bara me he. hamane hamari khoon bahahe.

Tokenized word: ['<s>', 'hello', '!', 'Ġye', 'Ġc', 'sci', 'Ġ5', '44', 'Ġka', 'Ġproject', 'Ġhe', '.', 'Ġye', 'Ġproject', 'Ġcode', '-', 'mix', 'ed', 'Ġmachine', 'Ġtrans', 'lation', 'Ġke', 'Ġbara', 'Ġme', 'Ġhe', '.', 'Ġhamane', 'Ġhamari', 'Ġkhoon', 'Ġbaha', 'he', '.', '</s>']

Tokenized idx: [0, 2318, 5, 521, 289, 21555, 467, 11343, 313, 3071, 357, 18, 521, 3071, 6439, 17, 19531, 518, 6263, 3398, 18403, 272, 17191, 285, 357, 18, 11715, 4631, 10624, 13968, 283, 18, 1]
####################################################################################################


In [14]:
token_idx = bart_tokenizer_append.encode(query, add_special_tokens=True)
token_word = bart_tokenizer_append.convert_ids_to_tokens(token_idx)
print('#'*100)
print("This is an example of the Bart Tokenizer to which we append our new data.\n")
print(f"Sentence: {query}\n")
print(f"Tokenized word: {token_word}\n")
print(f"Tokenized idx: {token_idx}")
print('#'*100)

####################################################################################################
This is an example of the Bart Tokenizer to which we append our new data.

Sentence: hello! ye csci 544 ka project he. ye project code-mixed machine translation ke bara me he. hamane hamari khoon bahahe.

Tokenized word: ['<s>', 'hello', '!', 'Ġ', 'ye', 'Ġ', 'cs', 'ci', 'Ġ', '54', '4', 'Ġ', 'ka', 'Ġ', 'project', 'Ġ', 'he', '.', 'Ġ', 'ye', 'Ġ', 'project', 'Ġ', 'code', '-', 'mix', 'ed', 'Ġ', 'mach', 'ine', 'Ġ', 'trans', 'lation', 'Ġ', 'ke', 'Ġ', 'bara', 'Ġ', 'me', 'Ġ', 'he', '.', 'Ġ', 'hamane', 'Ġ', 'hamari', 'Ġ', 'khoo', 'n', 'Ġ', 'baha', 'he', '.', '</s>']

Tokenized idx: [0, 42891, 328, 1437, 4717, 1437, 11365, 2520, 1437, 4283, 306, 1437, 2348, 1437, 28258, 1437, 700, 4, 1437, 4717, 1437, 28258, 1437, 20414, 12, 39915, 196, 1437, 51737, 833, 1437, 9981, 35019, 1437, 1071, 1437, 31533, 1437, 1794, 1437, 700, 4, 1437, 58472, 1437, 61134, 1437, 60709, 282, 1437, 56548, 700, 4, 2]
#######

In [15]:
# Now that we know how to get a BART-Tokenizer - default, appeded and scratch, let us get the tokenizer for our code-mixed language
# and the target language.
# Code-Mixed Language - Build a Bart-Tokenizer from scratch.
# Target Language (English) - Use the default Bart-Tokenizer.
hi_en_bart_tokenizer = CustomBartTokenizer().build(
    data=train_df["hi_en"],
    tokenizer_style=MBART_TOKENIZER_BPE_BINDING_BART_TOKENIZER_ENCODER_STYLE,
    tokenizer_bart_from_pretrained_path=MBART_TOKENIZER_BPE_BINDING_BART_TOKENIZER_ENCODER_FROM_PRETRAINED
)
en_bart_tokenizer = CustomBartTokenizer().build(
    tokenizer_style=MBART_TOKENIZER_BPE_BINDING_BART_TOKENIZER_DECODER_STYLE,
    tokenizer_bart_from_pretrained_path=MBART_TOKENIZER_BPE_BINDING_BART_TOKENIZER_DECODER_FROM_PRETRAINED
)

## Dataset and Data Loaders

In [16]:
# Get the data loaders for denoising
__data_loaders__ = get_data_loader_models(
    train_df=train_df,
    validation_df=validation_df,
    test_df=test_df,
    train_batch_size=4, 
    validation_batch_size=4, 
    test_batch_size=4,
    encoder_tokenizer=hi_en_bart_tokenizer,
    decoder_tokenizer=hi_en_bart_tokenizer,
    denoising_stage=True
)

In [17]:
__data_loaders__

defaultdict(dict,
            {'hi_en__en': {'object': <src.machine_translation.data.CodeMixedDataLoader at 0x16b5fd310>,
              'train': <torch.utils.data.dataloader.DataLoader at 0x16a3cd9d0>,
              'validation': <torch.utils.data.dataloader.DataLoader at 0x16a3cd7c0>,
              'test': <torch.utils.data.dataloader.DataLoader at 0x16a3cd6d0>}})

In [18]:
__data_loader__, __train_data_loader__, __validation_data_loader__, __test_data_loader__ = __data_loaders__["hi_en__en"].values()

In [19]:
__data_loader__.visualize()

####################################################################################################
Train Dataloader
Batch Size:  4
Number of batches:  43611
Batch source language shape:  torch.Size([4, 17])
Batch source language:  ['exercising shuru karne ke liye ek weekly reminder set kare', 'is week ke end tak meri passport application shuru karne ke liye reminder set kare', 'road ke halaath', 'amarillo aur happy ke beech ka traffic kaisa hai']
Batch source tokens:  tensor([[    0,  1845,  9566,   341,   653,   383,   272,   284,   380,  1217,
           348,   335,   316,     1,     2,     2,     2],
        [    0,   300,   423,   272,  1336,   374,   668, 12505,  6936,   653,
           383,   272,   284,   348,   335,   316,     1],
        [    0,  3246,   272,  5067,     1,     2,     2,     2,     2,     2,
             2,     2,     2,     2,     2,     2,     2],
        [    0, 18810, 14038,   369,  1899,   272,   777,   313,   375,   429,
           279,     1,     2,   

100%|█| 43611/43611 [00:41<00:00, 1049.7


Validation of train loader successful.
####################################################################################################
Val Dataloader
Batch Size:  4
Number of batches:  811
Validating validation laoder...


100%|█| 811/811 [00:30<00:00, 26.47it/s]


Validation of validation loader successful.
####################################################################################################
Test Dataloader
Batch Size:  4
Number of batches:  1768
Validating test laoder...


100%|█| 1768/1768 [00:31<00:00, 56.74it/

Validation of test loader successful.
####################################################################################################


In [20]:
# Get the data loaders for translation
__data_loaders__ = get_data_loader_models(
    train_df=train_df,
    validation_df=validation_df,
    test_df=test_df,
    train_batch_size=4, 
    validation_batch_size=4, 
    test_batch_size=4,
    encoder_tokenizer=hi_en_bart_tokenizer,
    decoder_tokenizer=en_bart_tokenizer
)

In [21]:
__data_loaders__

defaultdict(dict,
            {'hi_en__en': {'object': <src.machine_translation.data.CodeMixedDataLoader at 0x16ac64550>,
              'train': <torch.utils.data.dataloader.DataLoader at 0x16ac642b0>,
              'validation': <torch.utils.data.dataloader.DataLoader at 0x16ac640d0>,
              'test': <torch.utils.data.dataloader.DataLoader at 0x16ac64af0>}})

In [22]:
__data_loader__, __train_data_loader__, __validation_data_loader__, __test_data_loader__ = __data_loaders__["hi_en__en"].values()

In [23]:
__data_loader__.visualize()

####################################################################################################
Train Dataloader
Batch Size:  4
Number of batches:  43611
Batch source language shape:  torch.Size([4, 15])
Batch source language:  ['please mere sabhi timers cancel kardo', 'kya aaj raat baarish hogi', 'aj ke temperatures kya hai ?', 'team ko thursday subah 8 baje ki meeting ke baare me yaad dilaayen']
Batch source tokens:  tensor([[   0,  427,  377,  604, 1310,  606,  673,    1,    2,    2,    2,    2,
            2,    2,    2],
        [   0,  317,  440,  466,  708,  711,    1,    2,    2,    2,    2,    2,
            2,    2,    2],
        [   0,  352,  272, 3930,  355,  279,  319,    1,    2,    2,    2,    2,
            2,    2,    2],
        [   0, 6690,  281,  856,  481,  548,  396,  295,  796,  272,  632,  285,
          394,  669,    1]])
Batch source attention mask:  tensor([[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,

100%|█| 43611/43611 [00:42<00:00, 1028.2


Validation of train loader successful.
####################################################################################################
Val Dataloader
Batch Size:  4
Number of batches:  811
Validating validation laoder...


100%|█| 811/811 [00:31<00:00, 26.15it/s]


Validation of validation loader successful.
####################################################################################################
Test Dataloader
Batch Size:  4
Number of batches:  1768
Validating test laoder...


100%|█| 1768/1768 [00:31<00:00, 55.98it/

Validation of test loader successful.
####################################################################################################
